In [108]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K



import os
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [109]:
print(os.listdir("datasets/plantdisease-tugasakhir"))

['raw.txt', 'test', 'test.csv', 'train', 'train.csv']


In [110]:
dtrain = pd.read_csv("datasets/plantdisease-tugasakhir/train.csv")
dtest = pd.read_csv("datasets/plantdisease-tugasakhir/test.csv")

In [111]:
train_count = dtrain.cat.value_counts()
train_sum = sum(train_count)
print(train_count)
print('Data train total: ' + str(train_sum))

Apple___healthy                               1316
Grape___Esca_(Black_Measles)                  1107
Grape___Black_rot                              944
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)     861
Potato___Early_blight                          800
Potato___Late_blight                           800
Apple___Apple_scab                             504
Apple___Black_rot                              497
Grape___healthy                                339
Apple___Cedar_apple_rust                       220
Potato___healthy                               122
Name: cat, dtype: int64
Data train total: 7510


In [112]:
test_count = dtest.cat.value_counts()
test_sum = sum(test_count)
print(test_count)
print('Data train total: ' + str(test_sum))

Apple___healthy                               329
Grape___Esca_(Black_Measles)                  276
Grape___Black_rot                             236
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)    215
Potato___Early_blight                         200
Potato___Late_blight                          200
Apple___Apple_scab                            126
Apple___Black_rot                             124
Grape___healthy                                84
Apple___Cedar_apple_rust                       55
Potato___healthy                               30
Name: cat, dtype: int64
Data train total: 1875


In [113]:
print("Percentage of data train: {:.2f}%".format((train_sum / (train_sum + test_sum)*100)))
print("Percentage of data test: {:.2f}%".format((test_sum / (train_sum + test_sum)*100)))

Percentage of data train: 80.02%
Percentage of data test: 19.98%


In [121]:
dtrain.columns = ['category', 'image_name.jpg']

dtrain['category'][dtrain['category'] == 'Apple___Apple_scab'] = "(Apple) Apple Scab"
dtrain['category'][dtrain['category'] == 'Apple___Black_rot'] = "(Apple) Black Rot"
dtrain['category'][dtrain['category'] == 'Apple___Cedar_apple_rust'] = "(Apple) Cedar Apple Rust"
dtrain['category'][dtrain['category'] == 'Apple___healthy'] = "(Apple) Healthy"

dtrain['category'][dtrain['category'] == 'Grape___Black_rot'] = "(Grape) Black Rot"
dtrain['category'][dtrain['category'] == 'Grape___Esca_(Black_Measles)'] = "(Grape) Esca"
dtrain['category'][dtrain['category'] == 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'] = "(Grape) Leaf Blight"
dtrain['category'][dtrain['category'] == 'Grape___healthy'] = "(Grape) Healthy"

dtrain['category'][dtrain['category'] == 'Potato___Early_blight'] = "(Potato) Early Blight"
dtrain['category'][dtrain['category'] == 'Potato___Late_blight'] = "(Potato) Late Blight"
dtrain['category'][dtrain['category'] == 'Potato___healthy'] = "(Potato) Healthy"

In [124]:
dtest.columns = ['category', 'image_name.jpg']

dtest['category'][dtest['category'] == 'Apple___Apple_scab'] = "(Apple) Apple Scab"
dtest['category'][dtest['category'] == 'Apple___Black_rot'] = "(Apple) Black Rot"
dtest['category'][dtest['category'] == 'Apple___Cedar_apple_rust'] = "(Apple) Cedar Apple Rust"
dtest['category'][dtest['category'] == 'Apple___healthy'] = "(Apple) Healthy"

dtest['category'][dtest['category'] == 'Grape___Black_rot'] = "(Grape) Black Rot"
dtest['category'][dtest['category'] == 'Grape___Esca_(Black_Measles)'] = "(Grape) Esca"
dtest['category'][dtest['category'] == 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'] = "(Grape) Leaf Blight"
dtest['category'][dtest['category'] == 'Grape___healthy'] = "(Grape) Healthy"

dtest['category'][dtest['category'] == 'Potato___Early_blight'] = "(Potato) Early Blight"
dtest['category'][dtest['category'] == 'Potato___Late_blight'] = "(Potato) Late Blight"
dtest['category'][dtest['category'] == 'Potato___healthy'] = "(Potato) Healthy"

In [122]:
dtrain.sample(n=10)

,category,image_name.jpg
268,(Apple) Apple Scab,a0fb83fc-d5a2-4ddf-9a04-c2bf96e5027f___FREC_Sc...
6780,(Potato) Late Blight,22d5cf8c-308f-417d-9b05-b00a0791ecef___RS_LB 4...
3205,(Grape) Black Rot,c01ce5a9-a37d-4698-be6a-335868371a54___FAM_B.R...
7399,(Potato) Late Blight,e2ce6696-9f6e-445c-a6c6-c765d659447a___RS_LB 4...
7245,(Potato) Late Blight,c01daf14-9595-4b10-974d-3452fe133060___RS_LB 4...
4017,(Grape) Esca,985ada5a-091c-4c5d-bb88-a8ccc32a4d2c___FAM_B.M...
2544,(Grape) Black Rot,035ed95e-9c14-4096-8dce-21ae0970c6b6___FAM_B.R...
1844,(Apple) Healthy,8f38f758-8514-442b-82e9-90b53120db38___RS_HL 5...
5664,(Grape) Leaf Blight,e605eb27-faf6-4ac2-bd24-b7068e8ee2d6___FAM_L.B...
1438,(Apple) Healthy,32b6be5f-6482-4bd2-876d-c2a518afed09___RS_HL 6...


In [125]:
dtest.sample(n=10)

,category,image_name.jpg
274,(Apple) Cedar Apple Rust,3e4303ad-6293-4522-821b-ef5dd3b658ce___FREC_C....
1035,(Grape) Esca,4ccf1271-5da7-4951-88d2-c1d2edd3cd41___FAM_B.M...
541,(Apple) Healthy,5b264ecb-2c4c-47f3-93ec-c9f79a3e8e09___RS_HL 7...
921,(Grape) Esca,1a477ab0-0649-48d4-bbbf-fa3600d9ec03___FAM_B.M...
1488,(Potato) Early Blight,1ca9e3ba-e029-48bf-b207-e8cb3546f953___RS_Earl...
1502,(Potato) Early Blight,1f56ef5b-9a3b-4731-bf82-b08aeb84fcb5___RS_Earl...
1408,(Grape) Leaf Blight,6ee197ad-e554-4f70-b549-66d5f6d09d37___FAM_L.B...
84,(Apple) Apple Scab,6a2c0f48-b0f7-4bd9-b7b8-7410c1ef4048___FREC_Sc...
1587,(Potato) Early Blight,5bdc7f1d-26cc-45b2-b77f-e98084418fd9___RS_Earl...
164,(Apple) Black Rot,1e0244a0-b084-4f95-bf64-72efeef4dff2___JR_FrgE...


In [126]:
dtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7510 entries, 0 to 7509
Data columns (total 2 columns):
category          7510 non-null object
image_name.jpg    7510 non-null object
dtypes: object(2)
memory usage: 117.4+ KB


In [127]:
dtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 2 columns):
category          1875 non-null object
image_name.jpg    1875 non-null object
dtypes: object(2)
memory usage: 29.4+ KB


In [129]:
dtrain.dtypes

category          object
image_name.jpg    object
dtype: object

In [130]:
dtest.dtypes

category          object
image_name.jpg    object
dtype: object

In [2]:
x = "Hello world"
x[:2]

'He'

In [132]:
x[:-2]

'Hello wor'

In [133]:
x[-2:]

'ld'

In [134]:
x[2:-2]

'llo wor'

In [5]:
x[-3]

'r'

In [6]:
y = "00d8f10f-5038-4e0f-bb58-0b885ddc0cc5___RS_Early.B 8722"
z = "0c83302d-4233-4e98-8ecf-755a970495bb___RS_LB 4904"

In [35]:
y[42:][:-5]

'Early.B'

In [36]:
z[42:][:-5]

'LB'

In [37]:
cat = "cat.100"

In [38]:
cat[-3]

'1'